In [ ]:
!pip install pypdf
!pip install langchain
!pip install langchain-community
!pip install -U langchain-text-splitters
!pip install unstructured
!pip install pdf2image
!pip install pdfminer
!pip3 uninstall pdfminer.six -y
!pip3 install pdfminer.six
!pip install pillow-heif
!pip install pikepdf
!pip install sentence-transformers
!pip install chromadb

In [1]:
import os
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN']= HUGGINGFACEHUB_API_TOKEN

··········


# PDF Loader

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/pdfs/meta-earnings-report.pdf")
pages = loader.load_and_split()
print(pages[0].page_content)

Meta Reports Second  Quarter 2023  Results
MENLO PARK, Calif. – July 26, 2023  – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the 
quarter ended June 30, 2023 .
"We had a good quarter. We continue to see strong engagement across our apps and we have the most exciting roadmap I've 
seen in a while with Llama 2, Threads, Reels, new AI products in the pipeline, and the launch of Quest 3 this fall," said Mark 
Zuckerberg, Meta founder and CEO.
Second Quarter 2023  Financial Highlights
Three Months Ended June 30,
In millions, except percentages and per share amounts 2023 2022 % Change
Revenue $ 31,999 $ 28,822 11%
Costs and expenses  22,607  20,464 10%
Income from operations $ 9,392 $ 8,358 12%
Operating margin  29 %  29 %
Provision for income taxes $ 1,505 $ 1,499 —%
Effective tax rate  16 %  18 %
Net income $ 7,788 $ 6,687 16%
Diluted earnings per share (EPS) $ 2.98 $ 2.46 21%
Second Quarter 2023  Operational and Other Financial Highlights
•Family daily active 

# Store Embedding in ChromaDB

In [3]:
# import
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

In [4]:

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

print(len(docs))
docs[7].metadata

12


{'source': '/content/pdfs/meta-earnings-report.pdf', 'page': 6}

In [5]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
# load it into Chroma
import shutil

CHROMA_PATH = "/content/chroma/"
if os.path.exists(CHROMA_PATH):
  shutil.rmtree(CHROMA_PATH)

db = Chroma.from_documents(docs, embedding_function, persist_directory = CHROMA_PATH)
db.persist()
print(f"Save {len(docs)} chunks to {CHROMA_PATH}")


Save 12 chunks to /content/chroma/


In [7]:
# query it
query_text = "What is the Second Quarter 2023 Operational and Other Financial Highlights ? "
result = db.similarity_search(query_text,k=1)
result2 = db.similarity_search_with_relevance_scores(query_text,k=1)
# print(result2)
context_text = "\n\n---\n\n".join([doc.page_content for doc,score in result2])
context_text

'Meta Reports Second  Quarter 2023  Results\nMENLO PARK, Calif. – July 26, 2023  – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the \nquarter ended June 30, 2023 .\n"We had a good quarter. We continue to see strong engagement across our apps and we have the most exciting roadmap I\'ve \nseen in a while with Llama 2, Threads, Reels, new AI products in the pipeline, and the launch of Quest 3 this fall," said Mark \nZuckerberg, Meta founder and CEO.\nSecond Quarter 2023  Financial Highlights\nThree Months Ended June 30,\nIn millions, except percentages and per share amounts 2023 2022 % Change\nRevenue $ 31,999 $ 28,822 11%\nCosts and expenses  22,607  20,464 10%\nIncome from operations $ 9,392 $ 8,358 12%\nOperating margin  29 %  29 %\nProvision for income taxes $ 1,505 $ 1,499 —%\nEffective tax rate  16 %  18 %\nNet income $ 7,788 $ 6,687 16%\nDiluted earnings per share (EPS) $ 2.98 $ 2.46 21%\nSecond Quarter 2023  Operational and Other Financial Highlights\n•

# Prompt Template

In [8]:
PROMPT_TEMPLATE = """
Answer the question based only on the following contex:

{context}

---

Answer the question based on the above context:{query}
"""

In [13]:
# from langchain.prompts import ChatPromptTemplate
# q = "What is the Second Quarter 2023 Operational and Other Financial Highlights ? "

# context_text = "\n\n--\n\n".join([doc.page_content for doc,score in result2])
# prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# prompt = prompt_template.format(context = context_text ,query = q)


In [19]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(input_variables=['product'],
                        template="What is the {product}")
prompt

PromptTemplate(input_variables=['product'], template='What is the {product}')

# LLM

In [30]:
from langchain import HuggingFaceHub
llm = HuggingFaceHub(repo_id= 'mistralai/Mixtral-8x7B-Instruct-v0.1',
                     model_kwargs= {'temprature':0, 'max_length ': 500})


In [31]:
from langchain.chains import LLMChain
Chain = LLMChain(llm =llm , prompt = prompt)
Chain

LLMChain(prompt=PromptTemplate(input_variables=['product'], template='What is the {product}'), llm=HuggingFaceHub(client=<InferenceClient(model='mistralai/Mixtral-8x7B-Instruct-v0.1', timeout=None)>, repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1', task='text-generation', model_kwargs={'temprature': 0, 'max_length ': 500}))

In [33]:
result = Chain.invoke("Contacts to investors")
for key , value in result.items():
  print(f"{key}:{value}")

product:Contacts to investors
text:What is the Contacts to investors feature?

The Contacts to Investors feature allows you to send a message to all of your investors at once. This is a great way to keep your investors informed about your company's progress and to build a stronger relationship with them.

How do I use the Contacts to Investors feature?

To use the Contacts to Investors feature, follow these steps:

1. Log in to your StartEngine account.
2. Click on the "Camp


In [ ]:
sources = [doc.metadata.get("source",None ) for doc , _score in results2]
fprmated_response = f" Response: {response_text}\nSourses: {sources}"
print(formated_response)

In [34]:
print(llm.invoke("What is the Contacts to investors ?"))

What is the Contacts to investors ?

Contacts to investors is a list of investors who are interested in investing in your business. This list can include venture capitalists, angel investors, and other types of investors. Having a strong list of contacts to investors can be crucial for securing the funding you need to grow your business.

There are a few different ways to build a list of contacts to investors. One way is to attend networking events and conferences where investors are likely to be in attendance. This can be a


In [41]:
def predict(input, history):

  print(llm.invoke(input))
  response = llm.invoke(input)
  return response

In [ ]:
!pip install gradio

In [44]:
import gradio as gr

demo = gr.ChatInterface(fn=predict, title="Echo Bot")
demo.launch(share=True,debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3132ca9659f5907ec0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


What is the Segment Results ?

Segment Results is a financial statement that shows the financial performance of a company’s individual business segments. It is a required disclosure for publicly traded companies under Generally Accepted Accounting Principles (GAAP) and International Financial Reporting Standards (IFRS).

The Segment Results report typically includes the following information for each business segment:

1. Revenue: The amount of money earned from the sale of goods or services.
2. Cost of Sales:
What is the Second Quarter 2023 Financial Highlights?

The Second Quarter 2023 Financial Highlights for the company are not yet available as it is not the current reporting period. The most recent financial report available is for the First Quarter 2023, which was released on May 4, 2023. In that report, the company reported revenue of $2.2 billion, an increase of 12% year-over-year, and net income of $257 million, or $
What is the Second Quarter 2023 Operational and Other Financ